In [1]:
# import pandas as pd
# from pathlib import Path
# from tqdm import tqdm

In [2]:
# from transformers import pipeline

2023-03-23 10:28:46.963099: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [332]:
# Load three datasets: dataset contains all the data without labels, dataset_train_eval contains data with labels
# test contains data without labels
from datasets import load_dataset
dataset = load_dataset("csv",  data_files="../indexing/output.csv")
# We split the data test:eval 70:30 
dataset_train_eval = (load_dataset("csv", data_files="./balanced_labels_posneg.csv", split = "train").train_test_split(test_size=0.3))

test = load_dataset("csv", data_files="./test.csv")
# dataset["train"][100]

Found cached dataset csv (/Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-df29f072a3b1628c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-d0480d9a4aaa7bae/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


Found cached dataset csv (/Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-fe9974d2b3317d75/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [333]:
# Rename the column label to label_fine_grained [0,5]
# dataset_train_eval = dataset_train_eval.rename_column("label", "label_fine_grained")

In [353]:
# Rename the column label_posneg to label [0, 2]
dataset_train_eval = dataset_train_eval.rename_column("sentiment", "label")

ValueError: Original column name sentiment not in the dataset. Current columns in the dataset: ['Unnamed: 0', 'TEXT', 'label']

In [354]:
# Check the dataset_train_eval
dataset_train_eval["test"]

Dataset({
    features: ['Unnamed: 0', 'TEXT', 'label'],
    num_rows: 300
})

In [355]:
test

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'DATE', 'AUTHOR', 'TEXT', 'FAVORITE', 'year', 'manufacturer', 'model', 'Initial Index'],
        num_rows: 39347
    })
})

In [356]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [357]:
# Tokenizer: Hello, How are You: [Hello, How, Are, You]
# Specify Max Length 42 for now, the default model does not have a predefined maximum length
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)


def tokenize_function(examples):
    return tokenizer(examples["TEXT"], padding="max_length", truncation=True, max_length=42)


tokenized_datasets_train = dataset_train_eval['train'].map(tokenize_function, batched=True)
tokenized_datasets_eval = dataset_train_eval['test'].map(tokenize_function, batched=True)
tokenized_dataset_test = test['train'].map(tokenize_function, batched=True)

Loading cached processed dataset at /Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-d0480d9a4aaa7bae/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-1b3ca36310779346.arrow


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Loading cached processed dataset at /Users/parkhiagarwal/.cache/huggingface/datasets/csv/default-fe9974d2b3317d75/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-cde498592c4f1268.arrow


In [359]:
# We load the model, and specify the num_labels as 3 [pos, neg, neutral]
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2,ignore_mismatched_sizes=True)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

In [360]:
# Evaluation metrics, we can include more such as: F-measure
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

In [361]:
# import pytorch and check for GPU availability, for using torch.argmax in the future
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cpu


In [362]:
# Convert predictions to logits and use argmax function, need to change np.argmax to torch.argmax

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=predictions, references=labels, average = 'micro')["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels, average = 'micro')["recall"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall}

In [363]:
# Hyperparameter training, for now using default hyperparameters

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [364]:
# We can use optuna to do hyperparameter search
# def optuna_hp_space(trial):
#    return {
#        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
#        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128]),
#    }

In [365]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)

In [366]:
trainer.train()

/Users/parkhiagarwal/opt/anaconda3/envs/info/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,No log,0.346157,0.913333,0.913333,0.913333
2,No log,0.557495,0.906667,0.906667,0.906667
3,No log,0.684022,0.893333,0.893333,0.893333


TrainOutput(global_step=264, training_loss=0.2094721505136201, metrics={'train_runtime': 446.2253, 'train_samples_per_second': 4.706, 'train_steps_per_second': 0.592, 'total_flos': 45324990396000.0, 'train_loss': 0.2094721505136201, 'epoch': 3.0})

In [367]:
predictions = trainer.predict(tokenized_dataset_test)


In [368]:
trainer.evaluate()

{'eval_loss': 0.6840221285820007,
 'eval_accuracy': 0.8933333333333333,
 'eval_precision': 0.8933333333333333,
 'eval_recall': 0.8933333333333333,
 'eval_runtime': 16.4025,
 'eval_samples_per_second': 18.29,
 'eval_steps_per_second': 2.317,
 'epoch': 3.0}

In [369]:
print(predictions.predictions.shape)

(39347, 2)


In [370]:
preds = np.argmax(predictions.predictions, axis=-1)

In [384]:
for x in range(10):
    print(tokenized_dataset_test["TEXT"][x], "Prediction: ", preds[x])

It's a fun car to drive, great gas mileage, cute etc, but very expensive for repairs. In one year, I've spent $2500 out of pocket on a car I bought brand new from the dealer, in addition to the gas to get to the dealership (100 miles roundtrip in DC traffic) and taking time off work (they close at 7pm). Prediction:  0
My clutch went out at 14,000 miles, windows finicky (don't work all the time). Re the clutch: no explanation other than its my fault from Mini of North American and the dealer; I had to pay for the replacement less than a year after purchasing it. My previous clutches didn't burn out until 60,000-75,000 miles, so I don't understand this, but I won't buy another mini and will sell this one within the year. Prediction:  0
I love my British racing green convertible. The sidewalk package has all the bells and whistles esp the gorgeous brown leather interior. Now they won't make it any more. I am disappointed I wanted a new one only because I think the rear window visibility w

In [378]:
#metric = evaluate.load("glue", "mrpc")
#metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
#f1_metric = evaluate.load("f1")
#f1_metric.compute(predictions=preds, references=predictions.label_ids, average = 'micro')